In [1]:
from __future__ import division, absolute_import, print_function
%load_ext autoreload
%autoreload 2

import os 
#os.chdir(r"D:\data\ortoCC\design-with-alignments\mason_small")
print(os.getcwd()) 
    
import hepran
import hepran.bzipscore as bz
import hepran.bcipa as bc
import hepran.utils as u
import hepran.registers as r

import hepran.agadir as ag

import pandas as pd
import numpy as np
import seaborn as sns
    
%matplotlib inline

import matplotlib as mpl
import matplotlib.pylab as plt

from  score_utils import *
    
CCO = os.environ.get("CCO",r"C:\Projekti_KI\ortoCC\CoiledCoilOrtho")

D:\data\ortoCC\fine_tune_scoring_function
['f' 'g' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'a' 'b'
 'c' 'd' 'e' 'f' 'g' 'a' 'b' 'c' 'd' 'e' 'f']
Register_indices(a=array([ 2,  9, 16, 23], dtype=int64), b=array([ 3, 10, 17, 24], dtype=int64), c=array([ 4, 11, 18, 25], dtype=int64), d=array([ 5, 12, 19, 26], dtype=int64), e=array([ 6, 13, 20, 27], dtype=int64), f=array([ 0,  7, 14, 21, 28], dtype=int64), g=array([ 1,  8, 15, 22], dtype=int64), seq=array(['D', 'K', 'N', 'A', 'A', 'L', 'K', 'A', 'E', 'N', 'A', 'A', 'L',
       'E', 'Y', 'E', 'I', 'A', 'A', 'L', 'E', 'A', 'E', 'I', 'A', 'A',
       'L', 'E', 'G'], dtype='<U1'), reg=array(['f', 'g', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'a', 'b', 'c', 'd',
       'e', 'f', 'g', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'a', 'b', 'c',
       'd', 'e', 'f'], dtype='<U1'))


In [10]:
df = pd.read_excel('data/scores_Tm_mason_both.xlsx',index_col=[0,1])

In [11]:
df['seq1']=df['seq1'].str.replace('<tt>','')
df['seq1']=df['seq1'].str.replace('</tt>','')
df['seq2']=df['seq2'].str.replace('<tt>','')
df['seq2']=df['seq2'].str.replace('</tt>','')

In [12]:
df = df['seq1 seq2 Tm on_target'.split()]

In [13]:
df['avg_HP1']=df.seq1.apply(bc.get_HP)
df['avg_HP2']=df.seq2.apply(bc.get_HP)
df['avg_HP_12']=(df.avg_HP1+df.avg_HP2)/2

df['HP1']=df.seq1.apply(bc.get_HP, average=False)
df['HP2']=df.seq2.apply(bc.get_HP, average=False)
df['HP_12']=(df.HP1+df.HP2)

In [14]:
df.head(5)

seq1                           seq2  Tm  \
ID1 ID2                                                                     
O1  O1   DKNAALKAENAALEYEIAALEAEIAALEG  DKNAALKAENAALEYEIAALEAEIAALEG  -2   
    O2   DKNAALKAENAALEYEIAALEAEIAALEG  DENAALEAKNAALKYKIAALKAKIAALKG  72   
O2  O2   DENAALEAKNAALKYKIAALKAKIAALKG  DENAALEAKNAALKYKIAALKAKIAALKG  34   
O1  O3   DKNAALKAENAALEYEIAALEAEIAALEG  DEIAALEAKIAALKYENAALEAENAALEG   3   
O2  O3   DENAALEAKNAALKYKIAALKAKIAALKG  DEIAALEAKIAALKYENAALEAENAALEG  27   

         on_target   avg_HP1   avg_HP2  avg_HP_12    HP1    HP2  HP_12  
ID1 ID2                                                                 
O1  O1       False  1.286897  1.286897   1.286897  37.32  37.32  74.64  
    O2        True  1.286897  1.237241   1.262069  37.32  35.88  73.20  
O2  O2       False  1.237241  1.237241   1.237241  35.88  35.88  71.76  
O1  O3       False  1.286897  1.286897   1.286897  37.32  37.32  74.64  
O2  O3       False  1.237241  1.286897   1.262069  35.88  37.32  73.20

In [19]:
core_p, es_p = r.get_core_and_es_pairs('DKNAALKAENAALEYEIAALEAEIAALEG', 'DEIAALEAKNAALKYKIAALKAKIAALKG', 'f','f')
print(core_p)
r.pairs_counter(r.get_vertical_triplets(es_p), sort=False)

[('N', 'I'), ('N', 'N'), ('I', 'I'), ('I', 'I')]


Counter({'EEE': 2,
         'EEK': 1,
         'EKE': 1,
         'EKK': 2,
         'KEE': 2,
         'KEK': 1,
         'KKE': 1,
         'KKK': 2})

In [16]:
#core_p, es_p = r.get_core_and_es_pairs('DKNAALKAENAALEYEIAALEAEIAALEG', 'DEIAALEAKNAALKYKIAALKAKIAALKG', 'f','f')





df = df.apply(get_CC_features, axis=1)

In [17]:
df.columns

Index([               u'seq1',                u'seq2',                  u'Tm',
                 u'on_target',             u'avg_HP1',             u'avg_HP2',
                 u'avg_HP_12',                 u'HP1',                 u'HP2',
                     u'HP_12',          u'nterm_c_NN',          u'nterm_c_IN',
                u'nterm_c_II',          u'cterm_c_NN',          u'cterm_c_IN',
                u'cterm_c_II',                u'c_NN',                u'c_IN',
                      u'c_II',               u'es_EE',               u'es_EK',
                     u'es_KK',   u'interface_charge1',   u'interface_charge2',
       u'interface_repulsion',               u'cv_NN',               u'cv_II',
                     u'cv_NI',               u'cv_IN',              u'esv_KK',
                    u'esv_EE',              u'esv_EK',              u'esv_KE',
                   u'cvt_NNI',             u'cvt_IIN',             u'cvt_NNN',
                   u'cvt_NIN',             u'cvt_INN

In [162]:
df.head(50)

seq1                           seq2  Tm  \
ID1 ID2                                                                     
O1  O1   DKNAALKAENAALEYEIAALEAEIAALEG  DKNAALKAENAALEYEIAALEAEIAALEG  -2   
    O2   DKNAALKAENAALEYEIAALEAEIAALEG  DENAALEAKNAALKYKIAALKAKIAALKG  72   
O2  O2   DENAALEAKNAALKYKIAALKAKIAALKG  DENAALEAKNAALKYKIAALKAKIAALKG  34   
O1  O3   DKNAALKAENAALEYEIAALEAEIAALEG  DEIAALEAKIAALKYENAALEAENAALEG   3   
O2  O3   DENAALEAKNAALKYKIAALKAKIAALKG  DEIAALEAKIAALKYENAALEAENAALEG  27   
O3  O3   DEIAALEAKIAALKYENAALEAENAALEG  DEIAALEAKIAALKYENAALEAENAALEG  13   
O1  O4   DKNAALKAENAALEYEIAALEAEIAALEG  DKIAALKAEIAALEYKNAALKAKNAALKG  20   
O2  O4   DENAALEAKNAALKYKIAALKAKIAALKG  DKIAALKAEIAALEYKNAALKAKNAALKG  21   
O3  O4   DEIAALEAKIAALKYENAALEAENAALEG  DKIAALKAEIAALEYKNAALKAKNAALKG  68   
O4  O4   DKIAALKAEIAALEYKNAALKAKNAALKG  DKIAALKAEIAALEYKNAALKAKNAALKG  21   
O1  O5   DKNAALKAENAALEYEIAALEAEIAALEG  DENAALEAEIAALEYKNAALKAEIAALEG  21   
O2  O5   DENAALEAKNAALKYKIAALKAKIAALKG  DENAALEAEIAALEYKNAALKAEIAALEG  30   
O3  O5   DEIAALEAKIAALKYENAALEAENAALEG  DENAALEAEIAALEYKNAALKAEIAALEG  30   
O4  O5   DKIAALKAEIAALEYKNAALKAKNAALKG  DENAALEAEIAALEYKNAALKAEIAALEG  36   
O5  O5   DENAALEAEIAALEYKNAALKAEIAALEG  DENAALEAEIAALEYKNAALKAEIAALEG  17   
O1  O6   DKNAALKAENAALEYEIAALEAEIAALEG  DKNAALKAKIAALKYENAALEAKIAALKG  23   
O2  O6   DENAALEAKNAALKYKIAALKAKIAALKG  DKNAALKAKIAALKYENAALEAKIAALKG  30   
O3  O6   DEIAALEAKIAALKYENAALEAENAALEG  DKNAALKAKIAALKYENAALEAKIAALKG  33   
O4  O6   DKIAALKAEIAALEYKNAALKAKNAALKG  DKNAALKAKIAALKYENAALEAKIAALKG  38   
O5  O6   DENAALEAEIAALEYKNAALKAEIAALEG  DKNAALKAKIAALKYENAALEAKIAALKG  71   
O6  O6   DKNAALKAKIAALKYENAALEAKIAALKG  DKNAALKAKIAALKYENAALEAKIAALKG  23   
O1  O7   DKNAALKAENAALEYEIAALEAEIAALEG  DKIAALKAKNAALKYKIAALKAENAALEG  34   
O2  O7   DENAALEAKNAALKYKIAALKAKIAALKG  DKIAALKAKNAALKYKIAALKAENAALEG  15   
O3  O7   DEIAALEAKIAALKYENAALEAENAALEG  DKIAALKAKNAALKYKIAALKAENAALEG  18   
O4  O7   DKIAALKAEIAALEYKNAALKAKNAALKG  DKIAALKAKNAALKYKIAALKAENAALEG  16   
O5  O7   DENAALEAEIAALEYKNAALKAEIAALEG  DKIAALKAKNAALKYKIAALKAENAALEG  35   
O6  O7   DKNAALKAKIAALKYENAALEAKIAALKG  DKIAALKAKNAALKYKIAALKAENAALEG  19   
O7  O7   DKIAALKAKNAALKYKIAALKAENAALEG  DKIAALKAKNAALKYKIAALKAENAALEG  -1   
O1  O8   DKNAALKAENAALEYEIAALEAEIAALEG  DEIAALEAENAALEYEIAALEAKNAALKG  -3   
O2  O8   DENAALEAKNAALKYKIAALKAKIAALKG  DEIAALEAENAALEYEIAALEAKNAALKG  51   
O3  O8   DEIAALEAKIAALKYENAALEAENAALEG  DEIAALEAENAALEYEIAALEAKNAALKG  20   
O4  O8   DKIAALKAEIAALEYKNAALKAKNAALKG  DEIAALEAENAALEYEIAALEAKNAALKG  22   
O5  O8   DENAALEAEIAALEYKNAALKAEIAALEG  DEIAALEAENAALEYEIAALEAKNAALKG  26   
O6  O8   DKNAALKAKIAALKYENAALEAKIAALKG  DEIAALEAENAALEYEIAALEAKNAALKG  48   
O7  O8   DKIAALKAKNAALKYKIAALKAENAALEG  DEIAALEAENAALEYEIAALEAKNAALKG  68   
O8  O8   DEIAALEAENAALEYEIAALEAKNAALKG  DEIAALEAENAALEYEIAALEAKNAALKG   2   
O1  O9   DKNAALKAENAALEYEIAALEAEIAALEG  DENAALEAKNAALKYEIAALEAEIAALEG  12   
O2  O9   DENAALEAKNAALKYKIAALKAKIAALKG  DENAALEAKNAALKYEIAALEAEIAALEG  56   
O3  O9   DEIAALEAKIAALKYENAALEAENAALEG  DENAALEAKNAALKYEIAALEAEIAALEG  18   
O4  O9   DKIAALKAEIAALEYKNAALKAKNAALKG  DENAALEAKNAALKYEIAALEAEIAALEG  28   
O5  O9   DENAALEAEIAALEYKNAALKAEIAALEG  DENAALEAKNAALKYEIAALEAEIAALEG  11   
O6  O9   DKNAALKAKIAALKYENAALEAKIAALKG  DENAALEAKNAALKYEIAALEAEIAALEG  40   
O7  O9   DKIAALKAKNAALKYKIAALKAENAALEG  DENAALEAKNAALKYEIAALEAEIAALEG  17   
O8  O9   DEIAALEAENAALEYEIAALEAKNAALKG  DENAALEAKNAALKYEIAALEAEIAALEG  22   
O9  O9   DENAALEAKNAALKYEIAALEAEIAALEG  DENAALEAKNAALKYEIAALEAEIAALEG   6   
O1  O10  DKNAALKAENAALEYEIAALEAEIAALEG  DKNAALKAENAALEYKIAALKAKIAALKG  42   
O10 O2   DKNAALKAENAALEYKIAALKAKIAALKG  DENAALEAKNAALKYKIAALKAKIAALKG  46   
    O3   DKNAALKAENAALEYKIAALKAKIAALKG  DEIAALEAKIAALKYENAALEAENAALEG  10   
    O4   DKNAALKAENAALEYKIAALKAKIAALKG  DKIAALKAEIAALEYKNAALKAKNAALKG  32   
    O5   DKNAALKAENAALEYKIAALKAKIAALKG  DENAALEAEIAALEYKNAALKAEIAALEG  48   

         on_target   avg_HP1 

In [163]:
df.to_excel('data/round0_fiting.xlsx')